# HOW GEOGRAPHY, FERTILIZER AFFECT TO INDIAN CROPS

In [39]:
# Creating dependencies for setup
import matplotlib.pyplot as plt
import pandas as pd
import requests
import numpy as np
import warnings
from pathlib import Path
from scipy.stats import linregress

# Impor the OpenWeatherMap API key
from api_keys import weather_api_key

In [12]:
#Set file path
crop_csv=Path("resources/crop_yield.csv")
crop_csv

#Gather data from csv file
crop_df=pd.read_csv(crop_csv)

#CLean data
crop_cleaned_df=crop_df[["Crop","Crop_Year","State","Fertilizer","Pesticide","Yield"]]
crop_cleaned_df

,Crop,Crop_Year,State,Fertilizer,Pesticide,Yield
0,Arecanut,1997,Assam,7024878.38,22882.34,0.796087
1,Arhar/Tur,1997,Assam,631643.29,2057.47,0.710435
2,Castor seed,1997,Assam,75755.32,246.76,0.238333
3,Coconut,1997,Assam,1870661.52,6093.36,5238.051739
4,Cotton(lint),1997,Assam,165500.63,539.09,0.420909
...,...,...,...,...,...,...
19684,Small millets,1998,Nagaland,395200.00,1160.00,0.500000
19685,Wheat,1998,Nagaland,98800.00,290.00,3.000000
19686,Maize,1997,Jammu and Kashmir,29586735.11,96373.73,1.285000
19687,Rice,1997,Jammu and Kashmir,26242746.82,85481.26,0.016667


## GEOGRAPHY

In [50]:
# Set the API base URL
url = "http://api.openweathermap.org/data/2.5/weather?"

# Define an empty list to fetch the weather data for each city
states_name=crop_cleaned_df["State"].unique()
error_found=[]
state_data = []

# Print to logger
print("Starting to locate...")
print("-----------------------------")

appid=weather_api_key
#Loop through all state to retrieve data
for state in states_name:
    print("Getting the longitude and latitude of %s." % (state))
    params={"q": state,
           "appid":appid}
    try:
        response=requests.get(url,params=params).json()
        state_lat=response["coord"]["lat"]
        state_lon=response["coord"]["lon"]
        print("The coordinator of %s: Latitude: %s | Longitude: %s." % (state,state_lat,state_lon))
        state_data.append({"State":state,
                          "Lat":state_lat,
                          "Lon":state_lon})
        print("*****")
    except Exception as e:
        print("Uh oh, something went wrong. Error code: %s" % (e))
        error_found.append(state)
        print("*****")
print("------------------------------")
print("Finish coordinating. Data has been updated! \n %s error(s) found: (%s)." % (len(error_found),error_found))
print("------------------------------")

#As the coordinator for Telangana'
state_data

Starting to locate...
-----------------------------
Getting the longitude and latitude of Assam.
The coordinator of Assam: Latitude: 26 | Longitude: 93.
*****
Getting the longitude and latitude of Karnataka.
The coordinator of Karnataka: Latitude: 13.5 | Longitude: 76.
*****
Getting the longitude and latitude of Kerala.
The coordinator of Kerala: Latitude: 10 | Longitude: 76.5.
*****
Getting the longitude and latitude of Meghalaya.
The coordinator of Meghalaya: Latitude: 25.5 | Longitude: 91.3333.
*****
Getting the longitude and latitude of West Bengal.
The coordinator of West Bengal: Latitude: 24 | Longitude: 88.
*****
Getting the longitude and latitude of Puducherry.
The coordinator of Puducherry: Latitude: 11.9333 | Longitude: 79.8167.
*****
Getting the longitude and latitude of Goa.
The coordinator of Goa: Latitude: 15.3333 | Longitude: 74.0833.
*****
Getting the longitude and latitude of Andhra Pradesh.
The coordinator of Andhra Pradesh: Latitude: 16 | Longitude: 79.
*****
Getting

[{'State': 'Assam', 'Lat': 26, 'Lon': 93},
 {'State': 'Karnataka', 'Lat': 13.5, 'Lon': 76},
 {'State': 'Kerala', 'Lat': 10, 'Lon': 76.5},
 {'State': 'Meghalaya', 'Lat': 25.5, 'Lon': 91.3333},
 {'State': 'West Bengal', 'Lat': 24, 'Lon': 88},
 {'State': 'Puducherry', 'Lat': 11.9333, 'Lon': 79.8167},
 {'State': 'Goa', 'Lat': 15.3333, 'Lon': 74.0833},
 {'State': 'Andhra Pradesh', 'Lat': 16, 'Lon': 79},
 {'State': 'Tamil Nadu', 'Lat': 11, 'Lon': 78},
 {'State': 'Odisha', 'Lat': 20.5, 'Lon': 84.4167},
 {'State': 'Bihar', 'Lat': 47.15, 'Lon': 21.9167},
 {'State': 'Gujarat', 'Lat': 23, 'Lon': 72},
 {'State': 'Madhya Pradesh', 'Lat': 23.5, 'Lon': 78.5},
 {'State': 'Maharashtra', 'Lat': 19.5, 'Lon': 75},
 {'State': 'Mizoram', 'Lat': 23, 'Lon': 93},
 {'State': 'Punjab', 'Lat': 30.8602, 'Lon': 72.3198},
 {'State': 'Uttar Pradesh', 'Lat': 27.25, 'Lon': 80.75},
 {'State': 'Haryana', 'Lat': 29, 'Lon': 76},
 {'State': 'Himachal Pradesh', 'Lat': 31, 'Lon': 78},
 {'State': 'Tripura', 'Lat': 24, 'Lon': 9

In [35]:
#Get average yield for each state by Year
average_yield_states_years=crop_cleaned_df.groupby(["State"])["Yield"].mean().reset_index()

average_yield_states_years.sort_values(by="Yield",ascending=False)


,State,Yield
6,Goa,354.780305
21,Puducherry,346.512837
13,Kerala,276.611148
29,West Bengal,266.898311
24,Tamil Nadu,226.050221
2,Assam,219.716345
0,Andhra Pradesh,181.465391
12,Karnataka,105.720578
25,Telangana,99.518206
5,Delhi,13.118670


array([1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007,
       2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018,
       2019, 2020], dtype=int64)